In [2]:
import numpy as np
from typing import Callable
from numpy.typing import NDArray, ArrayLike
import csv
from datetime import datetime, timezone
import astropy
import pytz
from astropy.time import Time
import math
from pyquaternion import Quaternion
import pymap3d
import matplotlib.pyplot as plt
import ppigrf

In [3]:
def estimate_jacobian(f : Callable[NDArray[np.float64], NDArray[np.float64]],
                     state : NDArray[np.float64], #1 dimensional vector
                      epsilon = 1000 #limit definition of the derivative applied to vectorvalued partials
                     ):
    y = f(state)
    n = state.shape[0]#function input vector shape
    m = y.shape[0]#function output vector shape
    J = np.zeros((m, n))
    for i in range(n):
        state_1 = np.copy(state)
        state_1[i] = state[i] + epsilon
        f_1 = f(state_1)
        for j in range(m):
            J[j][i] = (f_1[j] - y[j])/epsilon

    return J

def raw_igdf_enu(height_from_sea_level : np.float64, lat : np.float64, lon : np.float64, time):
    #kilometers, degrees, degrees
    #returns B_east, b_north, b_up in nT: https://github.com/IAGA-VMOD/ppigrf/tree/main
    return ppigrf.igrf(lon, lat, height_from_sea_level, time)

def igdf_eci_vector(x : np.float64, y : np.float64, z : np.float64, time):
    #eci in METERS, returns unit eci igdf vector
    lat, lon, alt = pymap3d.eci2geodetic(x, y, z, time)
    b_east, b_north, b_up = raw_igdf_enu(alt/1000, lat, lon, time)
    ecefb_x, ecefb_y, ecefb_z = pymap3d.enu2ecef(b_east, b_north, b_up, lat, lon, alt)
    b_x, b_y, b_z = pymap3d.ecef2eci(ecefb_x, ecefb_y, ecefb_z, time)
    b_vec = np.array([b_x - x, b_y - y, b_z - z])
    return b_vec

In [138]:
#https://github.com/MartinAstro/GravityModels/tree/main

earth = Earth()
earth_sph_harm = SphericalHarmonics(earth.sh_file, 4)

In [139]:
air_density = 10e-15
body_length = 3
body_mass = 1

def df(state):
    position = np.array([state[0], state[1], state[2]], dtype = np.float64)
    a_g = earth_sph_harm.compute_acceleration(np.array([position]))[0]
    
    velocity = np.array([state[3], state[4], state[5]])
    drag_direction = -velocity/np.linalg.norm(velocity)
    drag_coefficient = air_density * body_length * state[6]
    drag_force = 1/2 * air_density * (np.linalg.norm(velocity)**2) * drag_coefficient
    drag_acceleration = drag_force/body_mass
    a_drag = drag_direction * drag_acceleration
    
    dr = velocity
    dv = a_g + a_drag
    return np.concatenate([dr, dv, np.array([0])])
def f(state, dt):
    return state + df(state)*dt

def h(state):
    time = datetime.now()
    magnetic_vec = igdf_eci_vector(state[0], state[1], state[2], time)
    return np.array([np.linalg.norm(magnetic_vec)])

In [140]:
#https://filterpy.readthedocs.io/en/latest/kalman/UnscentedKalmanFilter.html
from filterpy.kalman import UnscentedKalmanFilter, MerweScaledSigmaPoints
from filterpy.common import Q_discrete_white_noise

sigma_points = MerweScaledSigmaPoints(7, alpha=.1, beta=2., kappa=-1)
dt = 0.1
ukf = UnscentedKalmanFilter(dim_x=7, dim_z=1, dt=dt, fx=f, hx=h, points=sigma_points)

In [1]:
import autograd.numpy as np
from autograd import grad, jacobian

position_example = np.array([6.8e6, 0, 0])
magnetic_vec = -igdf_eci_vector(6.8e6, 0 ,0, datetime.now())
state_example = np.concatenate([position_example, np.ones(3), [1]], axis = 0)
state_example = np.array(state_example, dtype = np.float64)
z_variance = 300
f_now = lambda state: f(state, 1)
estimated_J = estimate_jacobian(f_now, state_example, 1)

ukf.x = state_example
ukf.P = np.linalg.inv(estimated_J.T@estimated_J)#http://ceres-solver.org/nnls_covariance.html
ukf.R = np.array([z_variance])

for i in range(1):
    ukf.predict()
    ukf.update(np.linalg.norm(magnetic_vec))

NameError: name 'igdf_eci_vector' is not defined